# Práctica 4: Clasificador de Bayes
Hacemos un clasificador de bayes para poder encontrar una prostata, su halo y el fondo del ultrasonido de forma automática. 

In [5]:
# Importamos la librerias que vamos a utilzar en la práctica
import cv2
import os
import numpy as np

from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join

## Carga de Imagenes
Cargamos las imagenes cortadas en tres secciones previamente:
1. Prostata: tiene la seccion del ultrasonido que tiene la prostata.
2. Halo: Carga el halo del utrasonido el cual es tejido que rodea la prostata.
3. Fondo: 

In [13]:
# Creamos tres listas en las que vamos a guardar las matrices de los ultrasonidos. 

prostatas = []
halos = []
fondos = []

mypath = (os.getcwd() + r'/imagenes/')

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file in onlyfiles:
    if "prost" in file:
        print(file)
        prostatas.append(cv2.imread(file,1))
    elif "halo" in file:
        halos.append(cv2.imread(file,1))
    elif "fondo" in file:
        fondos.append(cv2.imread(file,1))

prost-2.tiff
prost-3.tiff
prost-1.tiff


In [10]:
plt.imshow(fondos[1])
plt.show()

TypeError: Image data can not convert to float

In [12]:
type(fondos[1])

NoneType